# Sentiment Analysis App

## Intro
* Sentiment Analysis is a very popular functionality. For example, be able to determine if a product review is positive or negative.
* Our app will be able to do more than that. It will be a text classification app, also called a "tagging" app.
* In short, we will create an app to classify text into labels. And these labels can be:
    * Sentiment: Sentiment Analysis app.
    * Language: Language Analysis app.
    * Style (formal, informal, etc): Style Analysis app.
    * Topics or categories: Topic or category Analysis app.
    * Political tendency: Political Analysis app.
    * Etc.

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [3]:
#load our chat completion model
from langchain_groq import ChatGroq

llm = ChatGroq(model="mixtral-8x7b-32768")

## Tag Definition
* In the following code we define the 3 tags we will analize in this app:
    * sentiment.
    * political tendency.
    * language. 

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    political_tendency: str = Field(
        description="The political tendency of the user"
    )
    language: str = Field(description="The language the text is written in")


# LLM

llm = ChatGroq(temperature=0, model="mixtral-8x7b-32768").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

In [10]:
trump_follower = "I'm confident that President Trump's leadership and track record will once again resonate with Americans. His strong stance on economic growth and national security is exactly what our country needs at this pivotal moment. We need to bring back the proven leadership that can make America great again!"

In [11]:
biden_follower = "I believe President Biden's compassionate and steady approach is vital for our nation right now. His commitment to healthcare reform, climate change, and restoring our international alliances is crucial. It's time to continue the progress and ensure a future that benefits all Americans."

In [12]:
tagging_chain.invoke({"input": trump_follower})

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{"tool\\_call": {"id": "pending", "type": "function", "function": {"name": "Classification"}, "parameters": {"language": "English", "political\\_tendency": "Republican", "sentiment": "Positive"}}}\n</tool-use>'}}

In [13]:
tagging_chain.invoke({"input": biden_follower})

Classification(sentiment='Positive', political_tendency='Democratic', language='English')

* Careful schema definition gives us more control over the model's output.
* Specifically, we can define:
    * Possible values for each property.
    * Description to make sure that the model understands the property.
    * Required properties to be returned.
* Let's redeclare our Pydantic model to control for each of the previously mentioned aspects **using enums**:

In [14]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    political_tendency: str = Field(
        ...,
        description="The political tendency of the user",
        enum=["conservative", "liberal", "independent"],
    )
    language: str = Field(
        ..., enum=["spanish", "english"]
    )

In [16]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatGroq(temperature=0, model="mixtral-8x7b-32768").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

In [17]:
tagging_chain.invoke({"input": trump_follower})

Classification(sentiment='happy', political_tendency='conservative', language='english')

In [18]:
tagging_chain.invoke({"input": biden_follower})

Classification(sentiment='happy', political_tendency='liberal', language='english')

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-sentiment-analysis.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-sentiment-analysis.py